<a href="https://colab.research.google.com/github/ixd-ai-hub/Research-Ground/blob/project%2FCU-865d7myr5-shelf-object-detection/app/Shelf_Object_Detection_FinalizedV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up enviroment

In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install SAM model
%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
!pip install imagededup
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision
!pip install pillow==7.1.2

/content
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-y3b5j42d
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-y3b5j42d
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36586 sha256=0c6c6455100c407ccecde264f0f55a70cbe717747fb70d58a45cf89e78b88995
  Stored in directory: /tmp/pip-ephem-wheel-cache-_nhrs4im/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
# dowload SAM weights
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

/content
/content/weights


## Init the SAM model

In [ ]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

In [ ]:
#  Load Model
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)
#  Automatic Mask Generation
mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
import matplotlib.pyplot as plt
import cv2
from matplotlib.patches import Rectangle
from PIL import Image
from collections import defaultdict
import time

from imagededup.methods import CNN
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## build Object recongnizer

In [ ]:
def load_image(path):
  image_bgr = cv2.imread(path)
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  return image_rgb

In [ ]:
import json

def detect_objects(image, model, img_encoder, base_labels, base_feats, color_dict, thr=0.75):
  # generate segmentations
  start = time.perf_counter()
  sam_result = model.generate(image)
  print(f"Time taken for inference: {time.perf_counter()-start} milli. sec")

  annotations = []
  #plt.figure(figsize=(10,12))
  for c in sam_result:
    x,y,w,h = [int(i) for i in c['bbox']]

    # plt.text(int(x), int(y), str(c['predicted_iou'])[:4])
    f = image[y:y+h, x:x+w, :]
    # print(f.shape, x,y,w,h, image_rgb.shape)
    vec = img_encoder.encode_image(image_array=f)
    sims = cosine_similarity(base_feats, vec)
    lbl = np.argmax(sims)
    cls = labels[lbl]
    # print(f, sims[lbl], labels[lbl])
    if sims[lbl] > thr:
      #plt.gca().add_patch(Rectangle((x,y),w,h,linewidth=1,edgecolor=color_dict[cls],facecolor='none'))
      #plt.text(int(x), int(y), cls)
      annotations.append({
                "image_id": 1,
                "annotation_id": f"{len(annotations)+1:03}",
                "target_class": cls,
                "target_bbox": [x, y, x+w, y+h]
            })



  ##plt.imshow(image)
  #plt.show()
  return annotations


In [ ]:
base_path = "/content/drive/MyDrive/Shelf/Shelf"

In [ ]:
imgencoder = CNN()
encoding_map = imgencoder.encode_images(image_dir=base_path+'/Bases')

lbl_dict = defaultdict(list)
for k,v in encoding_map.items():
  lbl = k.split('-')[0]
  lbl_dict[lbl].append(v)
  # print(k)
  # labels.append(k.split('.')[0])
  # feats.append(v)

2023-09-11 11:11:11,025: INFO Device set to cpu ..
INFO:imagededup.methods.cnn:Device set to cpu ..
2023-09-11 11:11:11,032: INFO Initialized: mobilenet_v3_small for feature extraction ..
INFO:imagededup.methods.cnn:Initialized: mobilenet_v3_small for feature extraction ..
2023-09-11 11:11:11,338: INFO Start: Image encoding generation
INFO:imagededup.methods.cnn:Start: Image encoding generation
2023-09-11 11:11:18,081: INFO End: Image encoding generation
INFO:imagededup.methods.cnn:End: Image encoding generation


In [ ]:
lbl_dict['wax']

In [ ]:
labels, feats = [], []
for l, fs in lbl_dict.items():
  labels.append(l)
  feats.append(np.array(fs).mean(axis=0))

# np.array(lbl_dict['napkins']).mean(axis=0).shape

<ipython-input-17-63aa598549d1>:4: RuntimeWarning: Mean of empty slice.
  feats.append(np.array(fs).mean(axis=0))
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
color_dict = dict(zip(labels, ['b','r','g','w']))

In [ ]:
feats

In [ ]:
# imgencoder = CNN()
# encoding_map = imgencoder.encode_images(image_dir='/content/base/base')
# labels, feats = [], []
# for k,v in encoding_map.items():
#   labels.append(k.split('.')[0])
#   feats.append(v)
# feats = np.array(feats)

# color_dict = dict(zip(labels, ['b','r','g','w']))
# # color_dict

## Run

In [ ]:
IMAGE_NAME = base_path+"/Samples/shell2.jpg"
image_rgb = load_image(IMAGE_NAME)
annotations=detect_objects(image_rgb, mask_generator, imgencoder, labels, feats, color_dict, thr=0.76)

Time taken for inference: 628.9052740279999 milli. sec


In [ ]:
annotations

[{'image_id': 1,
  'annotation_id': '001',
  'target_class': 'napkins',
  'target_bbox': [183, 0, 264, 96]},
 {'image_id': 1,
  'annotation_id': '002',
  'target_class': 'tilecleaner',
  'target_bbox': [258, 0, 331, 120]},
 {'image_id': 1,
  'annotation_id': '003',
  'target_class': 'toiletpaper',
  'target_bbox': [374, 614, 455, 719]},
 {'image_id': 1,
  'annotation_id': '004',
  'target_class': 'toiletpaper',
  'target_bbox': [552, 154, 592, 224]},
 {'image_id': 1,
  'annotation_id': '005',
  'target_class': 'toiletpaper',
  'target_bbox': [324, 38, 386, 132]},
 {'image_id': 1,
  'annotation_id': '006',
  'target_class': 'toiletpaper',
  'target_bbox': [281, 628, 381, 719]},
 {'image_id': 1,
  'annotation_id': '007',
  'target_class': 'toiletpaper',
  'target_bbox': [597, 305, 631, 387]},
 {'image_id': 1,
  'annotation_id': '008',
  'target_class': 'tilecleaner',
  'target_bbox': [324, 0, 387, 143]},
 {'image_id': 1,
  'annotation_id': '009',
  'target_class': 'toiletpaper',
  'targe

#Downloading dataset from roboflow


In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="6SD3lhyA7O38SGwWUNX2")
project = rf.workspace("deeplearning-qyswt").project("supermercado-kuezz")
dataset = project.version(2).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...

In [ ]:
import os, glob,cv2
from collections import defaultdict

In [ ]:
def save_patch(patch, save_path, id, file_body):
  patch_path = f"{save_path}/{id}_{file_body}.jpg"
  if not os.path.isdir(save_path):
    os.makedirs(save_path)
  else:
    pass
  cv2.imwrite(patch_path, patch)

  return

In [ ]:
def bbox_formatter(bbox, format, img_height, img_width):
  if format == 'yolo-norm':
    center_x,center_y, bb_width, bb_height = bbox
    x, y, h, w = float(center_x)*img_width, float(center_y)*img_height, float(bb_height)*img_height, float(bb_width)*img_width
    crop_coords = int(y-h/2), int(y+h/2), int(x-w/2),int(x+w/2)
  # add any other format
  else:
    crop_coords = bbox
  return crop_coords


In [ ]:
def encoder(base_path):
  encoding_map = imgencoder.encode_images(image_dir=base_path+'/Bases')

  lbl_dict = defaultdict(list)
  for k,v in encoding_map.items():
    lbl = k.split('-')[0]
    lbl_dict[lbl].append(v)
  return lbl_dict

In [ ]:
def get_labels(lbl_dict):
  labels, feats = [], []
  for l, fs in lbl_dict.items():
    labels.append(l)
    feats.append(np.array(fs).mean(axis=0))
  color_dict = dict(zip(labels, ['b','r','g','w']))
  return labels,feats,color_dict

In [ ]:
print(os.path)

<module 'posixpath' from '/usr/lib/python3.10/posixpath.py'>


#**sample check for one Image**

In [ ]:
dataset_dir = "/content/weights/Supermercado-2/train/images/Image0_png.rf.318507b51f5d126fa0ccd286f2ad1130.jpg"

In [ ]:
imgencoder = CNN()
encoding_map = imgencoder.encode_images(image_dir=base_path+'/Bases')

lbl_dict = defaultdict(list)
for k,v in encoding_map.items():
  lbl = k.split('-')[0]
  lbl_dict[lbl].append(v)

2023-08-29 17:13:24,529: INFO Device set to cpu ..
INFO:imagededup.methods.cnn:Device set to cpu ..
2023-08-29 17:13:24,542: INFO Initialized: mobilenet_v3_small for feature extraction ..
INFO:imagededup.methods.cnn:Initialized: mobilenet_v3_small for feature extraction ..


ValueError: ignored

In [ ]:
color_dict = dict(zip(labels, ['b','r','g','w']))

NameError: ignored

In [ ]:
def detect_objectsS(image, model):
  # generate segmentations
  start = time.perf_counter()
  print("start")
  sam_result = model.generate(image)
  print(f"Time taken for inference: {time.perf_counter()-start} milli. sec")

In [ ]:
image_rgb = load_image(dataset_dir)

In [ ]:
detect_objectsS(image_rgb,mask_generator)

start


KeyboardInterrupt: ignored

#**INFERENCING**

In [ ]:
dataset_dir = "/content/weights/Supermercado-2/train/images"

In [ ]:
lbl_dict = encoder(base_path)

labels, feats, color_dict = get_labels(lbl_dict)

2023-09-06 12:32:00,181: INFO Start: Image encoding generation
INFO:imagededup.methods.cnn:Start: Image encoding generation
2023-09-06 12:32:00,518: INFO End: Image encoding generation
INFO:imagededup.methods.cnn:End: Image encoding generation


In [ ]:
import os
import cv2


output_dir = "/content/inferenced"

images_info = []

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for root, _, filenames in os.walk(dataset_dir):
    count=0
    annotations=[]
    images_info=[]
    for filename in filenames:
        if filename.endswith(".jpg"):
            count+=1
            images_info.append({
            "image_id": count,
            "image_path": dataset_dir+filename
            })
            image_path = os.path.join(root, filename)

            image_rgb = load_image(image_path)


            # Perform object detection using your detect_objects function
            annotation=detect_objects(image_rgb, mask_generator, imgencoder, labels, feats, color_dict, thr=0.76)
            annotations.append(annotation)

            # Save the annotated image directly to the /content directory
            #output_path = os.path.join(output_dir, filename)
            #cv2.imwrite(output_path, image_rgb

    result = {
    "images": images_info,
    "annotations": annotations
    }

    result_json = json.dumps(result, indent=4)
print(result_json)
print("Inference on all images complete.")

Time taken for inference: 593.8670176169999 milli. sec
Time taken for inference: 570.412361362 milli. sec
Time taken for inference: 590.5243396449996 milli. sec
Time taken for inference: 577.0538897870001 milli. sec
Time taken for inference: 585.1289331099997 milli. sec
Time taken for inference: 564.1990066449998 milli. sec


In [ ]:

print(result_json)


{
    "images": [
        {
            "image_id": 1,
            "image_path": "/content/weights/Supermercado-2/train/imagesImage432_png.rf.eb078b3958d45eebb0becb1e9a5934a9.jpg"
        }
    ],
    "annotations": [
        {
            "image_id": 1,
            "annotation_id": "001",
            "target_class": "toiletpaper",
            "target_bbox": [
                224,
                229,
                308,
                343
            ]
        },
        {
            "image_id": 1,
            "annotation_id": "002",
            "target_class": "wax",
            "target_bbox": [
                160,
                102,
                228,
                179
            ]
        },
        {
            "image_id": 1,
            "annotation_id": "003",
            "target_class": "wax",
            "target_bbox": [
                578,
                326,
                605,
                349
            ]
        },
        {
            "image_id": 1,
  